In [1]:
import nltk
import string
import random
import pandas as pd
from gtts import gTTS
import os
import pyaudio

from nltk.stem.porter import *
from nltk.stem import * 

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


from nltk.tokenize import RegexpTokenizer #regular expression tokenizer
import re  # regular expression module

import speech_recognition as sr

import sklearn
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer

Module #1: Load and Preprocess text data.
In the first module, go to https://en.wikipedia.org/wiki/Coronavirus_disease_2019 or to https://en.wikipedia.org/wiki/Tennis or pick any other Wikipedia text about a topic of your choice:
1. Copy some text from the page, 
2. Paste it in a word document, 
3. Save it as .txt file.
a. Save the file in the same directory where you saved your jupyter notebook for homework 3.
4. Preprocess and normalize your text.

In [2]:
## link used: https://en.wikipedia.org/wiki/Music

# 3)
File_object = open("Music_Wikipedia.txt","r")
file = File_object.read()
File_object.close() 
file

'Music is an art form, and a cultural activity, whose medium is sound. General definitions of music include common elements such as pitch (which governs melody and harmony), rhythm (and its associated concepts tempo, meter, and articulation), dynamics (loudness and softness), and the sonic qualities of timbre and texture (which are sometimes termed the "color" of a musical sound). Different styles or types of music may emphasize, de-emphasize or omit some of these elements. Music is performed with a vast range of instruments and vocal techniques ranging from singing to rapping; there are solely instrumental pieces, solely vocal pieces (such as songs without instrumental accompaniment) and pieces that combine singing and instruments. The word derives from Greek μουσική (mousike; "art of the Muses").[1] See glossary of musical terminology.\n\nIn its most general form, the activities describing music as an art form or cultural activity include the creation of works of music (songs, tunes,

In [3]:
#4) Lower case - no punkt - remove stop words - 

sent_tokens = sent_tokenize(file)

word_tokens = word_tokenize(file) 

stop_words = set(stopwords.words('english')) 

# ## remove stop words
new_file = []  

for w in word_tokens: 
    if w not in stop_words: 
        new_file.append(w) 


new_file1=" ".join(new_file)

## remove punctuation

no_punctuations="".join(s for s in new_file1 if s not in string.punctuation)

## make it lower case

final_file= no_punctuations.lower()

final_file

'music art form  cultural activity  whose medium sound  general definitions music include common elements pitch  governs melody harmony   rhythm  associated concepts tempo  meter  articulation   dynamics  loudness softness   sonic qualities timbre texture  sometimes termed  color  musical sound   different styles types music may emphasize  deemphasize omit elements  music performed vast range instruments vocal techniques ranging singing rapping  solely instrumental pieces  solely vocal pieces  songs without instrumental accompaniment  pieces combine singing instruments  the word derives greek μουσική  mousike   art muses     1  see glossary musical terminology  in general form  activities describing music art form cultural activity include creation works music  songs  tunes  symphonies    criticism music  study history music  aesthetic examination music  ancient greek indian philosophers defined music two parts  melodies  tones ordered horizontally  harmonies tones ordered vertically  

Module #2: Greeting the bot.
In the second module you will create two lists of strings, one is called: my_greetings and the other is called bot_response:
1. List: my_greetings consists of a selection of greeting words you may say to the bot.
2. List: bot_response consists of a selection of replies returned by the bot, randomly.
a. Upon your input, the bot checks if your input exists in my_greetings, if it does, then it picks a random reply from bot_response.
3. At any time, if you input an erroneous text, the bot should reply: “I am sorry, I do not understand what you are asking me.”

In [4]:
my_greetings= ['hi','hello','good afternoon','hola','bonjour','bonsoir','salut']
bot_response= ['Ahla w sahla','Hello there human! :)','Bonjour tout les jours!', 'Bonsoir tout les soirs!','Saba7o']

def greeting(user_input):
    boolean = 1
    
    for k in my_greetings:
        if k==user_input:
            index=random.randint(0, len(bot_response)-1)
            return bot_response[index]
            boolean = 0
    
    if boolean == 1:
        return "I am sorry, I do not understand what you are asking me."

Module #3: Text Vectorization and Similarity Measure.
    
In this third module, you will ask the bot some questions about the topic you chose in Module 1. For example, if you chose the tennis page on Wikipedia, then suitable questions can be:
• Tell me about Roger Federer
• What is a grand slam tournament?
In this case, the bot will search the text for an appropriate answer to your question. To that end, you should:
1. Vectorize your question and text using TF-IDF technique.
2. Using Cosine similarity measure, return from the text the part of text that has
the highest similarity measure with your question.
3. At any time, if you input an erroneous text, the bot should reply: “I am
sorry, I do not understand what you are asking me.”

### Please refer to the last cell to run all methods

In [5]:

# ===============THE BELOW DIDNT WORK===== TRIAL 1
# def ask_question():
#     question=input("What would you like to know?\n\n")
#    
#     feature_names = vectorizer.get_feature_names()
#     dense = vectors.todense()
#     denselist = dense.tolist()
#     df = pd.DataFrame(denselist, columns=feature_names)
    
#     cos_vec0=vectors[0].reshape(1,-1)
#     cos_vec1=vectors[1].reshape(1,-1)

#     cos_sim=cosine_similarity(vectors[0].reshape(1,-1),vectors[1].reshape(1,-1)) 
    
#     if cos_sim==0:
#         print("I am sorry, I do not understand what you are asking me.”)
#     else:
#         print (f"Cosine Similarity between the question and the text:{cos_sim}") 
#         print (f"Cosine Distance between the question and the text:{1-cos_sim}")


## ============ FINAL CODE ==============

vectorizer = TfidfVectorizer()

def ask_question(user_response):
    robo_response=''
    sent_tokens1=sent_tokenize(file)
    sent_tokens1.append(user_response)
    tfidf = vectorizer.fit_transform(sent_tokens1)
    
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+" I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens1[idx]
        return robo_response
    
def user_and_bot_text():
    bye = False
    print("Bobby Bot: My name is Bobby Bot, today I will answer all your questions on MUSIC! :)\nIf you want to end the conversation, type Bye or Bye Bobby!")
    print()
    while(bye==False):
        user_response = input()
        user_response=user_response.lower()  # to make sure it reads messages such as BONJOUR

        if(user_response!='bye'and user_response!='bye bobby'):
            bot_answer=greeting(user_response)
            print()
            print("Bobby Bot: "+bot_answer)
            print()
            question=input("Bobby Bot: Ask me something!\n\n")
            print("Bobby Bot: "+ ask_question(question))

        else:
            bye=True
            print("Bobby Bot: Bye Human! Take care :)")

Module #4: Speech Recognition.
In module 4, you will incorporate a speech recognition task.
Part I: To that end, you need to install on your system the SpeechRecognition package. Then:
1. Record your voice uttering a line of task such as a greeting: “Hello Bobby bot” or “What is Covid”, or “tell me about the tennis grand slam” (depending on your text).
2. Load the mp3 voice into your jupyter notebook
3. Input it to the speech recognizer
4. Print the text transcription of the line you recorded.
5. Feed this line to your bot, i.e., call the method with the transcribed text,
instead of user input (that you did in modules 2 and 3).
Part II: Then, use your microphone to talk directly and have the speech recognizer recognize your utterance. To that end you need to install:
1. portaudio
2. pyaudio
3. Installation instructions for Windows and Mac are found here:
https://people.csail.mit.edu/hubert/pyaudio/
4. The text you speak through the microphone, should be captured and sent to the bot, and the bot should reply (Module 3).


In [13]:
# Part 1

r = sr.Recognizer()
sound=sr.AudioFile("Greeting.wav")

with sound as source:
    r.adjust_for_ambient_noise(source)
    audio=r.listen(source)
    try:
        result=r.recognize_google(audio,language='en')
        print('Converting audio transcripts into text ...')
        print(result)

    except:
         print('Sorry.. run again...')
    

ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [6]:
# Part 2

def user():
    with sr.Microphone() as source:
        r=sr.Recognizer()
        r.adjust_for_ambient_noise(source)
        data=r.record(source,duration=10)
        print("User's voice......")

        try:
            text=r.recognize_google(data,language='en')
            return text
        except:
             print("Sorry, I did not get that")

                
def user_speaks():
    bye2=False
    print("Bobby Bot: My name is Bobby Bot, today I will answer all your questions on MUSIC! :)\nIf you want to end the conversation, type Bye or Bye Bobby!")
    print()

    while(bye2==False):
        print("Start talking now")
        user_response = user()
        user_response=user_response.lower()  # to make sure it reads messages such as BONJOUR

        if(user_response!='bye'and user_response!='bye bobby'):
            bot_answer=greeting(user_response)
            print()
            print("Bobby Bot: "+bot_answer)
            print()
            print("Bobby Bot: Ask me something!\n\n")
            print("Start talking now")
            question=user()
            print("Bobby Bot: "+ ask_question(question))

        else:
            bye2=True
            print("Bobby Bot: Bye Human! Take care :)")

Module #5:
In this fifth module, you need to implement a text-to-speech task, where you output the response of the bot through your system’s speakers. To this end you need to install:
1. gTTs package of pypi
2. Installation instructions for Windows and Mac are found here:
https://pypi.org/project/gTTS/

Module #6:
Finally, at any moment you type and/or speak “Bye” or “Bye Bobby”, the bot should reply with “Bye human” or “Take care” and close the input (you can no longer input text).

➔In fine, when I run your code, I should be able to greet your Bobby bot or ask him a question, by inputting a text and/or by speaking through the microphone. Bobby bot should reply to me by text and/or through the speakers.

➔All the best! Enjoy your first Chatbot!

In [7]:
def bot_speaks():
    bye5=False
    audio1 = gTTS(text="My name is Bobby Bot, today I will answer all your questions on MUSIC! :)\nIf you want to end the conversation, type Bye or Bye Bobby!", lang='en')
    audio1.save("message1.mp3")
    os.system("afplay message1.mp3")

    while(bye5==False):
        user_response = input()
        user_response=user_response.lower()  # to make sure it reads messages such as BONJOUR

        if(user_response!='bye'and user_response!='bye bobby'):
            bot_answer=greeting(user_response)

            audio2 = gTTS(text= bot_answer, lang='en')
            audio2.save("message2.mp3")
            os.system("afplay message2.mp3")

            audio3 = gTTS(text="Ask me something!\n\n", lang='en')
            audio3.save("message3.mp3")
            os.system("afplay message3.mp3")

            question=input()

            answer= ask_question(question)
            audio4 = gTTS(text=answer , lang='en')
            audio4.save("message4.mp3")
            os.system("afplay message4.mp3")

        else:
            bye5=True
            audio5 = gTTS(text="Bye Human! Take care! :)", lang='en')
            audio5.save("message5.mp3")
            os.system("afplay message5.mp3")

In [8]:
def user_and_bot_speak():
    bye6=False
    audio1 = gTTS(text="My name is Bobby Bot, today I will answer all your questions on MUSIC! :)\nIf you want to end the conversation, type Bye or Bye Bobby!", lang='en')
    audio1.save("message1.mp3")
    os.system("afplay message1.mp3")

    while(bye6==False):
        print("Start talking now")
        user_response = user()
        user_response=user_response.lower()  # to make sure it reads messages such as BONJOUR

        if(user_response!='bye'and user_response!='bye bobby'):
            bot_answer=greeting(user_response)

            audio2 = gTTS(text= bot_answer, lang='en')
            audio2.save("message2.mp3")
            os.system("afplay message2.mp3")
            
            audio3 = gTTS(text="Ask me something!\n\n", lang='en')
            audio3.save("message3.mp3")
            os.system("afplay message3.mp3")
            
            print("Start talking now")
            question=user()

            answer= ask_question(question)
            audio4 = gTTS(text=answer , lang='en')
            audio4.save("message4.mp3")
            os.system("afplay message4.mp3")

        else:
            bye6=True
            audio5 = gTTS(text="Bye Human! Take care! :)", lang='en')
            audio5.save("message5.mp3")
            os.system("afplay message5.mp3")

### What is music?
### What are some general definition of music?
### Where does the word music derive from?
### Where does music play a key role?
### Give me some examples of music genres

In [11]:
# =========== Please select any of the questions above =========

speaks=input("Would you like to speak or not? Enter yes or no ")
speaks=speaks.lower()
robot=input("Would you like to hear Bobby speak or not? Enter yes or no ")
robot=robot.lower()

if speaks=='yes':
    if robot=='yes':
        user_and_bot_speak()
        
    else: user_speaks()
    
elif robot=='yes':
    bot_speaks()
else: user_and_bot_text()

## NB: press stop and run if you want to run again or else it will get stuck for some reason
## ---------------------------------------------------------------------------------------------------------------

KeyboardInterrupt: 